In [42]:
from helpers import load_CSD_data, make_element_wise_environments

In [43]:
PATH_TRAIN = "CSD-2k_relaxed_shifts.txt"
PATH_TEST = "CSD-500.txt"
#structures_test, shifts_test = load_CSD_data(PATH_TEST,random_subsample=100)
structures_train, shifts_train = load_CSD_data(PATH_TRAIN,random_subsample=100)

In [44]:
from rascal.representations import SphericalInvariants

hypers = {"soap_type": "PowerSpectrum",
          "interaction_cutoff": 3,
          "radial_basis": "GTO",
          "max_radial": 9,
          "max_angular": 9,
          "gaussian_sigma_constant": 0.3,
          "gaussian_sigma_type":"Constant",
          "cutoff_function_type":"RadialScaling",
          "cutoff_smooth_width": 0.5,
          "normalize": True,
          "compute_gradients":False,
          "cutoff_function_parameters":dict(rate=1,scale=3.5,exponent=4)
          #"expansion_by_species_method":'structure wise'
         }

In [45]:
calculator = SphericalInvariants(**hypers)

In [47]:
H_environments_train, H_shifts_train = make_element_wise_environments(calculator,structures_train,shifts_train,select=6)

In [48]:
print("%d megabytes" % (H_environments_train.size/1e06 * H_environments_train.itemsize))

181 megabytes


In [49]:
H_shifts_train.shape

(2796,)

In [66]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics.pairwise import polynomial_kernel

import numpy as np

y = H_shifts_train

X = H_environments_train
clf = KernelRidge(alpha=1e-04,kernel='poly', degree=2)
clf.fit(X, y)

KernelRidge(alpha=0.0001, degree=2, kernel='poly')

In [52]:
#eval
structures_test, shifts_test = load_CSD_data(PATH_TEST)
H_environments_test, H_shifts_test = make_element_wise_environments(calculator,structures_test,shifts_test,select=6)

In [67]:
y_predicted = clf.predict(H_environments_test)

In [30]:
from sklearn.metrics import mean_squared_error

In [69]:
mean_squared_error(H_shifts_test[:,0], y_predicted,squared=False)

11.017001391306843

In [70]:
y_predicted

array([ 41.04934301,  41.04959007,  80.37122379, ..., 153.14477013,
       153.13863464, 153.13845142])

In [63]:
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process import GaussianProcessRegressor


gpr = GaussianProcessRegressor(random_state=0).fit(X, y)




In [64]:
y_gpr_predicted = gpr.predict(H_environments_test)

In [65]:
mean_squared_error(H_shifts_test[:,0], y_gpr_predicted,squared=False)

76.57197115964429